In [ ]:
from retinaface import RetinaFace
import cv2
import time

start = time.time()

resp = RetinaFace.detect_faces("./Test-Images/2.jpg")
img = cv2.imread("./Test-Images/1.jpg")
for key in resp.keys():
    face = resp[key]
    xmin = face['facial_area'][0]
    ymin = face['facial_area'][1]
    xmax = face['facial_area'][2]
    ymax = face['facial_area'][3]
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color=(255,0,0), thickness=2)

end = time.time()
print(end - start)

cv2.namedWindow("Detected Faces", cv2.WINDOW_NORMAL) 
  
cv2.resizeWindow("Detected Faces", 1000, 700) 

cv2.imshow("Detected Faces", img)
cv2.waitKey(0)

In [ ]:
from deepface import DeepFace
import matplotlib.pyplot as plt

faces = DeepFace.extract_faces(img_path = "./Test-Images/5.jpg", 
        target_size = (200, 200), 
        detector_backend = 'fastmtcnn'
)
# print(faces[0])
for face in faces:
    plt.imshow(face['face'])
    plt.show()

In [ ]:
#Create Temporary Database for the classroom

from deepface import DeepFace
from os import listdir
from os.path import isfile, join
import time
onlyfiles = [f for f in listdir('./Verification-Images') if isfile(join('./Verification-Images', f))]

database = []

start = time.time()
for file in onlyfiles:
    name = file.rstrip('.jpg')
    embedding = DeepFace.represent('./Verification-Images/{file}'.format(file=file), model_name = 'Facenet', align=True, detector_backend='retinaface', enforce_detection = False)
    temp = {
        'name': name,
        'face_embedding': embedding,
    }
    database.append(temp)
end = time.time()
database_creation_time = end - start
print('Time taken for creating face embedding database: {time} seconds'.format(time=database_creation_time))
print('Number of student information in the database: {number}'.format(number=len(database)))

In [4]:
# Create temporary database from JSON for the classroom

import json
from deepface import DeepFace
from retinaface import RetinaFace

import time

f = open('./session21-22/21-22.json')
database = json.load(f)
f.close()
i = 1
for student in database:
    image = student['image']
    embedding = DeepFace.represent(image, model_name = 'Facenet', align=True, detector_backend='retinaface', enforce_detection = False)
    student['face_embedding'] = embedding
    print(i)
    i+=1
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [2]:
!pip install tf-keras

  Using cached tensorflow-2.16.2-cp310-cp310-win_amd64.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 1.4 MB/s eta 0:00:02
    --------------------------------------- 0.0/1.7 MB 393.8 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.7 MB 939.4 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.7 MB 774.0 kB/s eta 0:00:03
   ----- ---------------------------------- 0.2/1.7 MB 1.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.7 MB 927.4 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.7 MB 1.1 MB/s eta 0:00:02
   -------- ------------------------------- 0.4/1.7 MB 1.1 MB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.7 MB 1.1 MB/s eta 0:00:02
   ------------- -------------------------- 0.6/1.7 MB 1.2 MB/s eta 0:00:01
   --------------- ------------------------ 0.7/1.7 MB 1.3 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 23.3.2 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from retinaface import RetinaFace
import matplotlib.pyplot as plt
from deepface import DeepFace
import numpy as np
from PIL import Image
import time
from deepface.modules import verification as vr

start_total = time.time()
path = "./session21-22/classroom/6.jpg"
threshold = vr.find_threshold('Facenet', 'euclidean')
faces = RetinaFace.extract_faces(img_path = path, align = True)
present_students = []

# resp = RetinaFace.detect_faces(path)
# img = cv2.imread(path)
# RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# for key in resp.keys():
#     face = resp[key]
#     xmin = face['facial_area'][0]
#     ymin = face['facial_area'][1]
#     xmax = face['facial_area'][2]
#     ymax = face['facial_area'][3]
#     cv2.rectangle(RGB_img, (xmin, ymin), (xmax, ymax), color=(255,0,0), thickness=4)
# plt.imshow(RGB_img)
# plt.show()

for face in faces:
    img = Image.fromarray(face)
    img.save('temp.jpg')
    
    start = time.time()
    embedding = np.array(DeepFace.represent('./temp.jpg', model_name = 'Facenet', align=True, detector_backend='retinaface', enforce_detection = False))
    for student in database:
        source = student['face_embedding'][0]['embedding']
        target = embedding[0]['embedding']
        distance = vr.find_euclidean_distance(source, target)
        if distance <= threshold:
            end = time.time()
            present_students.append(student)
            plt.imshow(face)
            plt.show()
            print('Recognised Student: {recognised_student} ({ID})'.format(recognised_student = student['name'], ID=student['ID']))
            print('Euclidean distance: {dst}'.format(dst=distance))
            print('Time taken in embedding and recognition: {time}'.format(time = end-start))
            break
    

end_total = time.time()
print("Total time: {time}".format(time=end_total-start_total))
print('Number of detected faces: {detected_faces}'.format(detected_faces=len(faces)))
print('Number of recognized students: {recognized_num}'.format(recognized_num=len(present_students)))


MemoryError: 

In [ ]:
!pip install imutils

In [ ]:
import cv2
import numpy as np
import imutils
from retinaface import RetinaFace
import matplotlib.pyplot as plt
from deepface import DeepFace
import numpy as np
from PIL import Image
import time
from deepface.modules import verification as vr

def beginRecognition():
    start_total = time.time()
    threshold = vr.find_threshold('Facenet', 'euclidean')
    faces = RetinaFace.extract_faces(img_path = "./snapshot.png", align = True)
    present_students = []

    for face in faces:
        img = Image.fromarray(face)
        img.save('temp.jpg')

        start = time.time()
        embedding = np.array(DeepFace.represent('./temp.jpg', model_name = 'Facenet', align=True, detector_backend='retinaface', enforce_detection = False))
        for student in database:
            source = student['face_embedding'][0]['embedding']
            target = embedding[0]['embedding']
            distance = vr.find_euclidean_distance(source, target)
            if distance <= threshold:
                end = time.time()
                present_students.append(student)
                plt.imshow(face)
                plt.show()
                print('Recognised Student: {recognised_student}'.format(recognised_student = student['name']))
                print('Euclidean distance: {dst}'.format(dst=distance))
                print('Time taken in embedding and recognition: {time}'.format(time = end-start))
                break


    end_total = time.time()
    print("Total time: {time}".format(time=end_total-start_total))
    print('Number of detected faces: {detected_faces}'.format(detected_faces=len(faces)))
    print('Number of recognized students: {recognized_num}'.format(recognized_num=len(present_students)))


url = "https://192.168.0.100:8080/video"
cap = cv2.VideoCapture(url)
captured = False

cv2.namedWindow("Stream", cv2.WINDOW_NORMAL)
cv2.resizeWindow('Stream', 900, 900) 
while(True):
    camera, frame = cap.read()
    if frame is not None:
        cv2.imshow("Stream", frame)
    q = cv2.waitKey(1)
    if q==ord("q"):
        break
    elif q%256 == 32:
        # SPACE pressed
        img_name = "snapshot.png"
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        captured = True
        break
cv2.destroyAllWindows()

if captured:
    beginRecognition()



In [2]:
import cv2
import numpy as np
import imutils
from retinaface import RetinaFace
import matplotlib.pyplot as plt
from deepface import DeepFace
import numpy as np
from PIL import Image
import time
from deepface.modules import verification as vr
import json


def inputStudentInformation():
    name = input("Student Name:")
    ID = input("Student ID:")

    url = "https://192.168.7.133:8080/video"
    cap = cv2.VideoCapture(url)
    captured = False
    cv2.namedWindow("Stream", cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Stream', 900, 900) 

    while(True):
        camera, frame = cap.read()
        if frame is not None:
            dispFrame = cv2.resize(frame, (200, 200))
            cv2.imshow("Stream", dispFrame)
        q = cv2.waitKey(1)
        if q==ord("q"):
            break
        elif q%256 == 32:
            # SPACE pressed
            img_name = "./session21-22/{filename}.png".format(filename=ID)
            cv2.imwrite(img_name, frame)
            break
    cv2.destroyAllWindows()
    temp = {
        'name': name,
        'ID': ID,
        'image': "./session21-22/{filename}.png".format(filename=ID)
    }

    f = open('./session21-22/21-22.json')
    training_data = json.load(f)
    training_data.append(temp)
    f.close()

    with open('./session21-22/21-22.json', 'w', encoding='utf-8') as f:
        json.dump(training_data, f, ensure_ascii=False, indent=4)
    f.close()
    print("Data entered successfully!")

try:
    while True:
        inputStudentInformation()
except KeyboardInterrupt:
    pass

Student Name:Sultanul Arefin
Student ID:22702037
Data entered successfully!
Student Name:Ayead Mujib Tamim
Student ID:22702035
Data entered successfully!
Student Name:Kollol Sarkar
Student ID:22702017
Data entered successfully!
Student Name:Abdullah Al Shakib
Student ID:22702059
Data entered successfully!
Student Name:Atikur Rahman
Student ID:22702049
Data entered successfully!
Student Name:Isabah Sharar
Student ID:22702057
Data entered successfully!
Student Name:Dibakor Roy
Student ID:22702054
Data entered successfully!
Student Name:Akram Hossain Ifty
Student ID:22702013
Data entered successfully!
Student Name:Afia Farzana
Student ID:22702018
Data entered successfully!
Student Name:Tasnim Tabassum Anika
Student ID:22702001
Data entered successfully!
Student Name:Tayaba Haque Chaity
Student ID:22702012
Data entered successfully!
Student Name:Zarifa Raiyan
Student ID:21702068
Data entered successfully!
Student Name:Mohammudul Islam Raihan
Student ID:22702005
Data entered successfully!
S